# Série TP 5 - Partie 1 - Fouille de Données - Neural Networks from Scratch

These steps will provide the foundation that you need to implement the backpropagation algorithm from scratch and apply it to your own predictive modeling problems.

Source : https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

## I - Backpropagation Algorithm Code 

### 1. Initialize Network and Data

In [1]:
# Initialize a network function - init random weight and bias values
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

### 2. Forward Propagation

We can break forward propagation down into three parts:
- Sum (= neuron input).
- Activation function  (= neuron output).
- Forward Propagation.

In [2]:
# Calculate neuron activation (sum) for an input : sum(weight_i * input_i) + bias

def activate_sum(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [3]:
# Activation function - Sigmoid : output = 1 / (1 + e^(-sum))

from math import exp
def sigmoid_activation(activation):
    return 1.0 / (1.0 + exp(-activation))

In [4]:
# Forward propagate input to a network output

def forward_propagate(network, row_data):
    inputs = row_data
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate_sum(neuron['weights'], inputs)
            print("Inputs : " + str(activation))
            neuron['output'] = sigmoid_activation(activation)
            print("Outputs : " + str(neuron['output']))
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

### 3. Back Propagate Error

This part is broken down into two sections.
- Transfer Derivative.
- Error Backpropagation.

In [5]:
# Calculate the derivative of an neuron output = output * (1.0 - output)
def transfer_derivative(output):
    return output * (1.0 - output)

In [6]:
# Backpropagate error and store in neurons
# output_error = (expected - output) * transfer_derivative(output)
# hidden_error = (weight_k * error_j) * transfer_derivative(output)

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
            print("Error : " + str(neuron['delta']))

### 4. Update  Network

In [7]:
# Update network weights with error
# weight = weight + learning_rate * error * input
# weight = weight + learning_rate * error

def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)): 
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
                print(neuron['delta'], neuron['weights'][j])                
            neuron['weights'][-1] += l_rate * neuron['delta']
            print(neuron['delta'], neuron['weights'][-1])

## II - Exemple : ANN vu en Cours 

### 1. Initialize Network and Data

In [8]:
# Initialization - 1 hidden layer (2 neurons) and 1 ontput layer (1 neuron)
network = [[{'weights': [0.2, 0.4, -0.5, -0.4]}, {'weights': [-0.3, 0.1, 0.2, 0.2]}],
           [{'weights': [-0.3, -0.2, 0.1]}]]

print(network)

[[{'weights': [0.2, 0.4, -0.5, -0.4]}, {'weights': [-0.3, 0.1, 0.2, 0.2]}], [{'weights': [-0.3, -0.2, 0.1]}]]


In [9]:
# data row and its class
row_data = [1, 0, 1, 1]
expected = [1]

### 2. Forward Propagation

In [10]:
# Forward propagation
output = forward_propagate(network, row_data)

Inputs : -0.7
Outputs : 0.3318122278318339
Inputs : 0.10000000000000003
Outputs : 0.52497918747894
Inputs : -0.10453950584533817
Outputs : 0.47388889882398544


### 3. Back Propagate Error

In [11]:
backward_propagate_error(network, expected)

Error : 0.1311690782143445
Error : -0.008724561965433263
Error : -0.006542085064168994


### 4. Update Network

In [12]:
l_rate = 0.9
print("Error \t\t\t New weight")
update_weights(network, row_data, l_rate)

Error 			 New weight
-0.008724561965433263 0.19214789423111006
-0.008724561965433263 0.4
-0.008724561965433263 -0.50785210576889
-0.008724561965433263 -0.40785210576888997
-0.006542085064168994 -0.30588787655775207
-0.006542085064168994 0.1
-0.006542085064168994 0.19411212344224793
-0.006542085064168994 0.19411212344224793
0.1311690782143445 -0.26082884634154524
0.1311690782143445 -0.13802506750700472
0.1311690782143445 0.21805217039291008


In [13]:
network

[[{'weights': [0.19214789423111006,
    0.4,
    -0.50785210576889,
    -0.40785210576888997],
   'output': 0.3318122278318339,
   'delta': -0.008724561965433263},
  {'weights': [-0.30588787655775207,
    0.1,
    0.19411212344224793,
    0.19411212344224793],
   'output': 0.52497918747894,
   'delta': -0.006542085064168994}],
 [{'weights': [-0.26082884634154524,
    -0.13802506750700472,
    0.21805217039291008],
   'output': 0.47388889882398544,
   'delta': 0.1311690782143445}]]

## III - Exemple : ANN vu en TD 

In [14]:
# Initialization - 1 hidden layer (2 neurons) and 1 ontput layer (2 neurons)
network = [[{'weights': [0.15, 0.20, 0.35]}, {'weights': [0.25, 0.30, 0.35]}],
           [{'weights': [0.40, 0.45, 0.60]}, {'weights': [0.50, 0.55, 0.60]}]]

print(network)

[[{'weights': [0.15, 0.2, 0.35]}, {'weights': [0.25, 0.3, 0.35]}], [{'weights': [0.4, 0.45, 0.6]}, {'weights': [0.5, 0.55, 0.6]}]]


In [15]:
# Data row and its class
row_data = [0.05, 0.10, 0.01]
expected = [0.01, 0.99]

In [16]:
# Forward propagation 
output = forward_propagate(network, row_data)

Inputs : 0.3775
Outputs : 0.5932699921071872
Inputs : 0.39249999999999996
Outputs : 0.596884378259767
Inputs : 1.1059059670597702
Outputs : 0.7513650695523157
Inputs : 1.2249214040964653
Outputs : 0.7729284653214625


In [17]:
# Error Backpropagation
backward_propagate_error(network, expected)

Error : -0.13849856162855698
Error : 0.03809823651655623
Error : -0.008771354689486937
Error : -0.009954254705217202


In [18]:
# Update weights
l_rate = 0.9
print("Error \t\t\t New weight")
update_weights(network, row_data, l_rate)

Error 			 New weight
-0.008771354689486937 0.14960528903897308
-0.008771354689486937 0.19921057807794618
-0.008771354689486937 0.34210578077946174
-0.009954254705217202 0.24955205853826523
-0.009954254705217202 0.29910411707653045
-0.009954254705217202 0.3410411707653045
-0.13849856162855698 0.32604966349219233
-0.13849856162855698 0.3755991349372201
-0.13849856162855698 0.4753512945342987
0.03809823651655623 0.5203422864297276
0.03809823651655623 0.5704662179943805
0.03809823651655623 0.6342884128649006


In [19]:
network

[[{'weights': [0.14960528903897308, 0.19921057807794618, 0.34210578077946174],
   'output': 0.5932699921071872,
   'delta': -0.008771354689486937},
  {'weights': [0.24955205853826523, 0.29910411707653045, 0.3410411707653045],
   'output': 0.596884378259767,
   'delta': -0.009954254705217202}],
 [{'weights': [0.32604966349219233, 0.3755991349372201, 0.4753512945342987],
   'output': 0.7513650695523157,
   'delta': -0.13849856162855698},
  {'weights': [0.5203422864297276, 0.5704662179943805, 0.6342884128649006],
   'output': 0.7729284653214625,
   'delta': 0.03809823651655623}]]

## IV - Training a NN and Make Predictions

### Code

In [20]:
# Train a network for a fixed number of epochs (iterations)

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
        print('========================')

In [21]:
# Make a prediction with a network

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

### Example

In [22]:
# Training a network with 2 inputs, 2 neurons hidden layer, and 2 neuron output layer
from random import seed
from random import random
seed(1)

train_dataset = [[2.7810836, 2.550537003, 0],
            [1.465489372, 2.362125076, 0],
            [3.396561688, 4.400293529, 0],
            [8.675418651, -0.242068655, 1],
            [7.673756466, 3.508563011, 1]]

n_inputs = len(train_dataset[0]) - 1
n_outputs = len(set([row[-1] for row in train_dataset]))

network = initialize_network(n_inputs, 2, n_outputs)

n_epoch = 1

l_rate = 0.9

train_network(network, train_dataset, l_rate, n_epoch, n_outputs)

Inputs : 3.2988639183529895
Outputs : 0.9643898158763548
Inputs : 2.422484871353145
Outputs : 0.9185258960543243
Inputs : 1.4467120366053836
Outputs : 0.8094918973879515
Inputs : 1.227776976537604
Outputs : 0.7734292563511262
Error : 0.029379162360871295
Error : -0.13553299084477086
Error : 0.000525476610401129
Error : -0.0067428584755714034
0.000525476610401129 0.13567949905743437
0.000525476610401129 0.8486399597223678
0.000525476610401129 0.764247547925975
-0.0067428584755714034 0.23819181792824232
-0.0067428584755714034 0.4799569680487976
-0.0067428584755714034 0.4434224921607239
0.029379162360871295 0.6770926412045849
0.029379162360871295 0.8130103204250735
0.029379162360871295 0.12030083289901906
-0.13553299084477086 -0.08928849595535791
-0.13553299084477086 0.7237235982453156
-0.13553299084477086 0.3107873761447596
Inputs : 2.967678141148764
Outputs : 0.9510923872578426
Inputs : 1.9262084594609177
Outputs : 0.8728291594590579
Inputs : 1.473897604025162
Outputs : 0.81364907992209

In [23]:
network

[[{'weights': [0.0025495762749162144, 0.85001491182507, 0.7499848856979607],
   'output': 0.9798684915008027,
   'delta': -0.002050753565075248},
  {'weights': [0.17604491878324185, 0.4467606839861707, 0.42777757448083115],
   'output': 0.9686583727742528,
   'delta': -0.0013046701278474856}],
 [{'weights': [0.5175364369641436, 0.6469661077918362, -0.055881603066407604],
   'output': 0.8035756596451186,
   'delta': -0.12683784371887416},
  {'weights': [-0.1781997027521733, 0.6532413269845773, 0.2310361908188606],
   'output': 0.6099556785752978,
   'delta': 0.09279534651114298}]]

In [24]:
# Test making predictions with the network

test_dataset = [[2.7810836, 2.550537003, 0],
                [1.465489372, 2.362125076, 0],
                [7.627531214,2.759262235, 1],
                [5.332441248, 2.088626775, 1]]

for row in test_dataset:
    prediction = predict(network, row)
    print('\nExpected=%d, Got=%d' % (row[-1], prediction))
    print("===============")

Inputs : 2.9250699561747027
Outputs : 0.9490719111443161
Inputs : 2.056852866964555
Outputs : 0.886638233521248
Inputs : 1.008922579210643
Outputs : 0.7328092433258854
Inputs : 0.641100594583188
Outputs : 0.6550022091235739

Expected=0, Got=0
Inputs : 2.761562800827881
Outputs : 0.9405630613133313
Inputs : 1.7410741465669206
Outputs : 0.8508234506551131
Inputs : 0.9813479887141392
Outputs : 0.7273756048678184
Inputs : 0.6192211728086938
Outputs : 0.6500413957715643

Expected=0, Got=0
Inputs : 3.1148459037031286
Outputs : 0.9575009871615809
Inputs : 3.0032955709719134
Outputs : 0.9527227878498435
Inputs : 1.0560394000786477
Outputs : 0.7419329422942285
Inputs : 0.6827676977052319
Outputs : 0.6643561386502865

Expected=1, Got=0
Inputs : 2.5389442553783512
Outputs : 0.9268272598273708
Inputs : 2.29964308749223
Outputs : 0.9088474753317881
Inputs : 1.0117787883577343
Outputs : 0.7333681166624069
Inputs : 0.6595725794273319
Outputs : 0.6591643678138522

Expected=1, Got=0
